In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import pandas as pd

# Step 1: Copy your Google Sheet ID
sheet_id = "1tZgqv4JIsIL_orhMGsjvYak8yubM50GiA1P45TWJ_fs"

# Step 2: Create a CSV export URL
sheet_name = "Sheet1"  # Change if your sheet has a different name
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

# Step 3: Load it into a DataFrame
df = pd.read_csv(url)

df.head(4)


/usr/local/lib/python3.12/dist-packages/google/colab/_dataframe_summarizer.py:88: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  cast_date_col = pd.to_datetime(column, errors="coerce")


,Unnamed: 0,MMM-YY,Driver_ID,Age,Gender,City,Education_Level,Income,Dateofjoining,LastWorkingDate,Joining Designation,Grade,Total Business Value,Quarterly Rating
0,0,01/01/19,1,28.0,0.0,C23,2,57387,24/12/18,NaN,1,1,2381060,2
1,1,02/01/19,1,28.0,0.0,C23,2,57387,24/12/18,NaN,1,1,-665480,2
2,2,03/01/19,1,28.0,0.0,C23,2,57387,24/12/18,03/11/19,1,1,0,2
3,3,11/01/20,2,31.0,0.0,C7,2,67016,11/06/20,NaN,2,2,0,1


In [3]:
!pip install category_encoders


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 3.6 MB/s eta 0:00:00


In [4]:
# ===============================
# 📦 1. Import Required Libraries
# ===============================
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from imblearn.over_sampling import SMOTE
from category_encoders import TargetEncoder

from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
import lightgbm as lgb

# ===================================
# 🧩 2. Load Data from Google Sheets
# ===================================
sheet_id = "1tZgqv4JIsIL_orhMGsjvYak8yubM50GiA1P45TWJ_fs"
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet=Sheet1"
df = pd.read_csv(url)

print("✅ Data loaded successfully!")
print(df.shape)
print(df.info())

# ======================================
# 🧹 3. Basic Cleaning & Type Conversions
# ======================================
df['MMM-YY'] = pd.to_datetime(df['MMM-YY'], errors='coerce')
df['Dateofjoining'] = pd.to_datetime(df['Dateofjoining'], errors='coerce')
df['LastWorkingDate'] = pd.to_datetime(df['LastWorkingDate'], errors='coerce')

# Create target variable
df['Churn'] = df['LastWorkingDate'].notnull().astype(int)

# ======================================
# 🩺 4. Handle Missing Values (KNN Imputer)
# ======================================
imputer = KNNImputer(n_neighbors=5)
df[['Age','Gender']] = imputer.fit_transform(df[['Age','Gender']])

# ======================================
# 🔢 5. Feature Engineering
# ======================================
# Tenure in years
df['EndDate'] = df['LastWorkingDate'].fillna(df['MMM-YY'])
df['Tenure_Years'] = (df['EndDate'] - df['Dateofjoining']).dt.days / 365

# Flags
df['High_Business_Value_Flag'] = (df['Total Business Value'] >= df['Total Business Value'].quantile(0.90)).astype(int)
df['Low_Income_Flag'] = (df['Income'] <= df['Income'].quantile(0.10)).astype(int)
df['Senior_Driver_Flag'] = (df['Age'] > 50).astype(int)
df['Recent_Joiner_Flag'] = (df['Tenure_Years'] < 1).astype(int)
df['Low_Rating_Flag'] = (df['Quarterly Rating'] <= 2).astype(int)

# Binning Age
bins = [0, 30, 50, df['Age'].max()]
labels = ['Young', 'Middle-aged', 'Senior']
df['Age_Group'] = pd.cut(df['Age'], bins=bins, labels=labels, include_lowest=True)

# ======================================
# 🧮 6. Drop Unnecessary Columns
# ======================================
cols_to_drop = ['Unnamed: 0','Driver_ID','MMM-YY','Dateofjoining',
                'LastWorkingDate','year','month','EndDate']
df = df.drop(columns=[c for c in cols_to_drop if c in df.columns])

# ======================================
# 🔄 7. Train-Test Split
# ======================================
X = df.drop(columns=['Churn'])
y = df['Churn']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# ======================================
# 🎯 8. Encode Categorical Features
# ======================================
# Target encode City
target_enc = TargetEncoder(cols=['City'])
X_train['City'] = target_enc.fit_transform(X_train['City'], y_train)
X_test['City'] = target_enc.transform(X_test['City'])

# Label encode Age_Group
le = LabelEncoder()
X_train['Age_Group'] = le.fit_transform(X_train['Age_Group'])
X_test['Age_Group'] = le.transform(X_test['Age_Group'])

# ======================================
# ⚖️ 9. Handle Class Imbalance with SMOTE
# ======================================
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)
print("SMOTE completed. Balanced data:", np.bincount(y_resampled))

# ======================================
# 🚀 10. Model Training and Tuning
# ======================================

# Random Forest
rf_params = {'n_estimators':[100,200],'max_depth':[10,20],'max_features':['sqrt','log2']}
rf_grid = GridSearchCV(RandomForestClassifier(class_weight='balanced', random_state=42),
                       rf_params, scoring='f1', cv=5, n_jobs=-1)
rf_grid.fit(X_resampled, y_resampled)
rf_pred = rf_grid.best_estimator_.predict(X_test)

# Bagging
base_tree = DecisionTreeClassifier(class_weight='balanced', random_state=42)
bag_params = {'n_estimators':[50,100],'max_samples':[0.5,1.0],'max_features':[0.5,1.0],
              'estimator__max_depth':[5,10,None],'estimator__min_samples_split':[2,5]}
bag_grid = GridSearchCV(BaggingClassifier(estimator=base_tree, random_state=42),
                        bag_params, scoring='f1', cv=5, n_jobs=-1)
bag_grid.fit(X_resampled, y_resampled)
bag_pred = bag_grid.best_estimator_.predict(X_test)

# Gradient Boosting
gb_params = {'n_estimators':[100,200],'learning_rate':[0.05,0.1],'max_depth':[3,5]}
gb_grid = GridSearchCV(GradientBoostingClassifier(random_state=42),
                       gb_params, scoring='f1', cv=5, n_jobs=-1)
gb_grid.fit(X_resampled, y_resampled)
gb_pred = gb_grid.best_estimator_.predict(X_test)

# XGBoost
xgb_params = {'n_estimators':[100,200],'learning_rate':[0.05,0.1],
              'max_depth':[3,5],'scale_pos_weight':[1,2]}
xgb_grid = GridSearchCV(xgb.XGBClassifier(random_state=42, eval_metric='logloss'),
                        xgb_params, scoring='f1', cv=5, n_jobs=-1)
xgb_grid.fit(X_resampled, y_resampled)
xgb_pred = xgb_grid.best_estimator_.predict(X_test)

# LightGBM
lgb_params = {'n_estimators':[100,200],'learning_rate':[0.05,0.1],'max_depth':[3,5]}
lgb_grid = GridSearchCV(lgb.LGBMClassifier(class_weight='balanced', random_state=42),
                        lgb_params, scoring='f1', cv=5, n_jobs=-1)
lgb_grid.fit(X_resampled, y_resampled)
lgb_pred = lgb_grid.best_estimator_.predict(X_test)

# ======================================
# 📊 11. Compare Model Performance
# ======================================
def evaluate(name, y_true, y_pred):
    return {
        "Model": name,
        "Accuracy": accuracy_score(y_true, y_pred),
        "Precision": precision_score(y_true, y_pred, zero_division=0),
        "Recall": recall_score(y_true, y_pred),
        "F1 Score": f1_score(y_true, y_pred)
    }

results = [
    evaluate("Random Forest", y_test, rf_pred),
    evaluate("Bagging", y_test, bag_pred),
    evaluate("Gradient Boosting", y_test, gb_pred),
    evaluate("XGBoost", y_test, xgb_pred),
    evaluate("LightGBM", y_test, lgb_pred)
]

results_df = pd.DataFrame(results).sort_values(by="F1 Score", ascending=False)
print("📈 Model Comparison:\n", results_df)
print(f"\n✅ Recommended Best Model: {results_df.iloc[0]['Model']}")


✅ Data loaded successfully!
(19104, 14)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19104 entries, 0 to 19103
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            19104 non-null  int64  
 1   MMM-YY                19104 non-null  object 
 2   Driver_ID             19104 non-null  int64  
 3   Age                   19043 non-null  float64
 4   Gender                19052 non-null  float64
 5   City                  19104 non-null  object 
 6   Education_Level       19104 non-null  int64  
 7   Income                19104 non-null  int64  
 8   Dateofjoining         19104 non-null  object 
 9   LastWorkingDate       1616 non-null   object 
 10  Joining Designation   19104 non-null  int64  
 11  Grade                 19104 non-null  int64  
 12  Total Business Value  19104 non-null  int64  
 13  Quarterly Rating      19104 non-null  int64  
dtypes: float64(2), int64(8), objec

/tmp/ipython-input-297766893.py:35: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['MMM-YY'] = pd.to_datetime(df['MMM-YY'], errors='coerce')
/tmp/ipython-input-297766893.py:36: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Dateofjoining'] = pd.to_datetime(df['Dateofjoining'], errors='coerce')
/tmp/ipython-input-297766893.py:37: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['LastWorkingDate'] = pd.to_datetime(df['LastWorkingDate'], errors='coerce')


SMOTE completed. Balanced data: [13990 13990]
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 13990, number of negative: 13990
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002840 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1562
[LightGBM] [Info] Number of data points in the train set: 27980, number of used features: 16
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -

In [6]:
# Mount your Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Import joblib to save model and encoders
import joblib
import os

# Define save path
save_dir = '/content/drive/MyDrive/ola_driver_churn_project/models'
os.makedirs(save_dir, exist_ok=True)

# Save the trained XGBoost model and encoders
joblib.dump(xgb_grid.best_estimator_, f'{save_dir}/xgboost_final_model.pkl')
joblib.dump(le, f'{save_dir}/label_encoder.pkl')
joblib.dump(target_enc, f'{save_dir}/target_encoder.pkl')

print("✅ Model and encoders saved successfully to Google Drive!")


Mounted at /content/drive
✅ Model and encoders saved successfully to Google Drive!
